In [1]:
import pandas as pd
from my_libs import lib_tools as pt

run_gridSearchCV = True  # True to run hyperparameters optimization with GridSearchCV()
run_optuna = True        # True to run hyperparameters optimization with Optuna

run_type = 'very-light'
# run_type = 'light'
# run_type = 'full'

if run_type == 'very-light': filename_train, filename_test = 'df-very-light-train.pkl', 'df-very-light-test.pkl'
if run_type == 'light'     : filename_train, filename_test = 'df-light-train.pkl', 'df-light-test.pkl'
if run_type == 'full'      : filename_train, filename_test = 'df-full-train.pkl', 'df-full-test.pkl'

# classifier_name = 'DecisionTreeClassifier'
# classifier_name = 'RandomForestClassifier'
classifier_name = 'GradientBoostingClassifier'

columns = ['catv', 'agg', 'dep', 'col', 'catr', 'catu', 'trajet', 'locp', 'circ', 'situ', 'lum', 'age_cls']
X_train, y_train, X_test, y_test, X_test_final, y_test_final = pt.get_train_valid_test_data(filename_train, filename_test, columns)

In [2]:
import time
from my_libs.encoder_custom import EncoderCustom

cols_target_encoded = ['dep']
cols_onehot_encoded = X_train.columns.drop(cols_target_encoded)

encoder = EncoderCustom(cols_target_encoded=cols_target_encoded, cols_onehot_encoded=cols_onehot_encoded)
X_train, y_train = encoder.transform(X_train, y_train, 'Train')
X_test, y_test = encoder.transform(X_test,  y_test,  'Test')
X_test_final, y_test_final = encoder.transform(X_test_final, y_test_final, 'Test')

C:\Users\Nicolas\anaconda3\lib\site-packages\category_encoders\target_encoder.py:122: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
C:\Users\Nicolas\anaconda3\lib\site-packages\category_encoders\target_encoder.py:127: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


Classes cardinality after resampling :
0    4887
1    4887
Name: grav, dtype: int64
X shape : (9774, 12)
Columns target encoded : ['dep']
Columns one hot encoded : ['catv', 'agg', 'col', 'catr', 'catu', 'trajet', 'locp', 'circ', 'situ', 'lum', 'age_cls']
Features normalized
--- Train set - features encoding performed in 5.97 seconds ---
--- Test set - features encoding performed in 0.09 seconds ---
--- Test set - features encoding performed in 0.13 seconds ---


In [3]:
import time
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

start_time = time.time()

if classifier_name == 'DecisionTreeClassifier':
    model = DecisionTreeClassifier()
    params = {'max_depth' : [2, 10, 30, 50],
              'min_samples_split' : [2,4,6]
              }

if classifier_name == 'RandomForestClassifier':
    model = RandomForestClassifier()
    params = {'n_estimators' : [5,10,25,50],
              'criterion' : ('gini', 'entropy'),
    }

if classifier_name == 'GradientBoostingClassifier':
    model = GradientBoostingClassifier()
    params = {'learning_rate' : [0.001, 0.01, 0.5],
              'n_estimators' : [50, 100, 200]
              }

grid = GridSearchCV(estimator=model, param_grid=params, cv = 3, verbose=2, scoring="f1")
grid.fit(X_train, y_train)

print(f"\n--- {classifier_name} - Optimization with GridSearchCV performed in %s seconds ---" % (time.time() - start_time))
print(f"Grid search params : {params}")
print(f"Best params : {grid.best_params_}")

from my_libs.model_evaluator import ModelEvaluator
evaluator = ModelEvaluator(model_type=classifier_name, params=grid.best_params_, X_train=X_train, y_train=y_train, X_test=X_test_final, y_test=y_test_final)
model = evaluator.evaluate()

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] END ...............learning_rate=0.001, n_estimators=50; total time=   1.0s
[CV] END ...............learning_rate=0.001, n_estimators=50; total time=   1.1s
[CV] END ...............learning_rate=0.001, n_estimators=50; total time=   0.9s
[CV] END ..............learning_rate=0.001, n_estimators=100; total time=   1.6s
[CV] END ..............learning_rate=0.001, n_estimators=100; total time=   1.6s
[CV] END ..............learning_rate=0.001, n_estimators=100; total time=   1.6s
[CV] END ..............learning_rate=0.001, n_estimators=200; total time=   4.5s
[CV] END ..............learning_rate=0.001, n_estimators=200; total time=   4.1s
[CV] END ..............learning_rate=0.001, n_estimators=200; total time=   3.9s
[CV] END ................learning_rate=0.01, n_estimators=50; total time=   1.1s
[CV] END ................learning_rate=0.01, n_estimators=50; total time=   1.0s
[CV] END ................learning_rate=0.01, n_es

Classe prédite,0,1
Classe réelle,,
0,785,864
1,103,247



Classification report :
              precision    recall  f1-score   support

           0       0.88      0.48      0.62      1649
           1       0.22      0.71      0.34       350

    accuracy                           0.52      1999
   macro avg       0.55      0.59      0.48      1999
weighted avg       0.77      0.52      0.57      1999



In [4]:
feats = {}
for feature, importance in zip(X_train.columns, grid.best_estimator_.feature_importances_):
    feats[feature] = importance

importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Gini-importance'})

# variables les plus importantes
importances.sort_values(by='Gini-importance', ascending=False).head(20)

,Gini-importance
dep,0.343201
catu_2,0.073812
catr_1,0.063786
catv_2,0.043995
trajet_3,0.036727
agg_1,0.031174
catv_3,0.025417
col_1,0.023241
circ_1,0.022421
lum_3,0.019317


In [5]:
from my_libs.model_evaluator import ModelEvaluator

evaluator = ModelEvaluator(model_type=classifier_name, params=grid.best_params_, X_train=X_train, y_train=y_train, X_test=X_test_final, y_test=y_test_final)
model = evaluator.evaluate()


--- Model GradientBoostingClassifier fit and trained in 6.450602293014526 seconds ---
--- Params : {'learning_rate': 0.5, 'n_estimators': 200}


Classe prédite,0,1
Classe réelle,,
0,769,880
1,102,248



Classification report :
              precision    recall  f1-score   support

           0       0.88      0.47      0.61      1649
           1       0.22      0.71      0.34       350

    accuracy                           0.51      1999
   macro avg       0.55      0.59      0.47      1999
weighted avg       0.77      0.51      0.56      1999

